In [1]:
# set up main path where everything will be you should download the
# hugging face directory described in readme and put it here on the same
# server where the data analyzer is run so that the data analyzer code with 
# the GPU can access these files
# You should replace the below path with your location
import json
import os
try:
    with open('../conf/config.json', 'r') as f:
        config = json.load(f)
    storage_path = config['storage_path']
    data_path = config['data_path']
    derivatives_path = config['derivatives_path']
    fsl_path = config['fsl_path']
    project_path = config['project_path']
    assert os.path.exists(storage_path), "The specified storage path does not exist."
    assert os.path.exists(data_path), "The specified data path does not exist."
    assert os.path.exists(derivatives_path), "The specified derivatives path does not exist."
    assert os.path.exists(fsl_path), "The specified FSL path does not exist."
    assert os.path.exists(project_path), "The specified project path does not exist."
except FileNotFoundError:
    raise FileNotFoundError("config.json file not found. Please create it with the required paths.")

"""-----------------------------------------------------------------------------
Imports and set up for mindEye
-----------------------------------------------------------------------------"""

import sys
import shutil
import argparse
import numpy as np
import math
import time
import random
import string
import h5py
from scipy import stats
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import transforms
from accelerate import Accelerator, DeepSpeedPlugin
# SDXL unCLIP requires code from https://github.com/Stability-AI/generative-models/tree/main
sys.path.append(f'{project_path}/models/generative_models')
sys.path.append(f'{project_path}/models')
import sgm
from generative_models.sgm.modules.encoders.modules import FrozenOpenCLIPImageEmbedder, FrozenOpenCLIPEmbedder2
from generative_models.sgm.models.diffusion import DiffusionEngine
from generative_models.sgm.util import append_dims
from omegaconf import OmegaConf
from PIL import Image
# tf32 data type is faster than standard float32
torch.backends.cuda.matmul.allow_tf32 = True
# custom functions #
import utils_mindeye
from models import *
import pandas as pd
import ants
import nilearn
import pdb
from nilearn.plotting import plot_design_matrix
### Multi-GPU config ###
local_rank = os.getenv('RANK')
if local_rank is None: 
    local_rank = 0
else:
    local_rank = int(local_rank)
accelerator = Accelerator(split_batches=False, mixed_precision="fp16")
device = accelerator.device

%load_ext autoreload
%autoreload 2

/teamspace/studios/this_studio/rtcloud-projects/mindeye/conf/.venv/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


line 6:  /teamspace/studios/this_studio/rtcloud-projects/mindeye/scripts
line 6:  /teamspace/studios/this_studio/rtcloud-projects/mindeye/scripts
line 14:  /teamspace/studios/this_studio/rtcloud-projects/mindeye/scripts
line 14:  /teamspace/studios/this_studio/rtcloud-projects/mindeye/scripts


In [2]:
if accelerator.mixed_precision == "bf16":
    data_type = torch.bfloat16
elif accelerator.mixed_precision == "fp16":
    data_type = torch.float16
else:
    data_type = torch.float32

In [3]:
# cache_dir= f"{storage_path}/cache"
# model_name="sub-005_ses-01_task-C_bs24_MST_rishab_repeats_3split_0_avgrepeats_finalmask"
# subj=1
# hidden_dim=1024
# blurry_recon = False
# n_blocks=4 
# seq_len = 1

# import pickle
# with open(f"{storage_path}/clip_img_embedder", "rb") as input_file:
#     clip_img_embedder = pickle.load(input_file)
# clip_img_embedder.to(device)
# clip_seq_dim = 256
# clip_emb_dim = 1664

In [4]:
# class MindEyeModule(nn.Module):
#     def __init__(self):
#         super(MindEyeModule, self).__init__()
#     def forward(self, x):
#         return x

# model = MindEyeModule()

# class RidgeRegression(torch.nn.Module):
#     # make sure to add weight_decay when initializing optimizer
#     def __init__(self, input_sizes, out_features, seq_len): 
#         super(RidgeRegression, self).__init__()
#         self.out_features = out_features
#         self.linears = torch.nn.ModuleList([
#                 torch.nn.Linear(input_size, out_features) for input_size in input_sizes
#             ])
#     def forward(self, x, subj_idx):
#         out = torch.cat([self.linears[subj_idx](x[:,seq]).unsqueeze(1) for seq in range(seq_len)], dim=1)
#         return out
# num_voxels = 2792 # 8627
# model.ridge = RidgeRegression([num_voxels], out_features=hidden_dim, seq_len=seq_len)

# from diffusers.models.vae import Decoder
# class BrainNetwork(nn.Module):
#     def __init__(self, h=4096, in_dim=15724, out_dim=768, seq_len=2, n_blocks=n_blocks, drop=.15, 
#                 clip_size=768):
#         super().__init__()
#         self.seq_len = seq_len
#         self.h = h
#         self.clip_size = clip_size

#         self.mixer_blocks1 = nn.ModuleList([
#             self.mixer_block1(h, drop) for _ in range(n_blocks)
#         ])
#         self.mixer_blocks2 = nn.ModuleList([
#             self.mixer_block2(seq_len, drop) for _ in range(n_blocks)
#         ])

#         # Output linear layer
#         self.backbone_linear = nn.Linear(h * seq_len, out_dim, bias=True) 
#         self.clip_proj = self.projector(clip_size, clip_size, h=clip_size)


#     def projector(self, in_dim, out_dim, h=2048):
#         return nn.Sequential(
#             nn.LayerNorm(in_dim),
#             nn.GELU(),
#             nn.Linear(in_dim, h),
#             nn.LayerNorm(h),
#             nn.GELU(),
#             nn.Linear(h, h),
#             nn.LayerNorm(h),
#             nn.GELU(),
#             nn.Linear(h, out_dim)
#         )

#     def mlp(self, in_dim, out_dim, drop):
#         return nn.Sequential(
#             nn.Linear(in_dim, out_dim),
#             nn.GELU(),
#             nn.Dropout(drop),
#             nn.Linear(out_dim, out_dim),
#         )

#     def mixer_block1(self, h, drop):
#         return nn.Sequential(
#             nn.LayerNorm(h),
#             self.mlp(h, h, drop),  # Token mixing
#         )

#     def mixer_block2(self, seq_len, drop):
#         return nn.Sequential(
#             nn.LayerNorm(seq_len),
#             self.mlp(seq_len, seq_len, drop)  # Channel mixing
#         )

#     def forward(self, x):
#         # make empty tensors
#         c,b,t = torch.Tensor([0.]), torch.Tensor([[0.],[0.]]), torch.Tensor([0.])

#         # Mixer blocks
#         residual1 = x
#         residual2 = x.permute(0,2,1)
#         for block1, block2 in zip(self.mixer_blocks1,self.mixer_blocks2):
#             x = block1(x) + residual1
#             residual1 = x
#             x = x.permute(0,2,1)

#             x = block2(x) + residual2
#             residual2 = x
#             x = x.permute(0,2,1)

#         x = x.reshape(x.size(0), -1)
#         backbone = self.backbone_linear(x).reshape(len(x), -1, self.clip_size)
#         c = self.clip_proj(backbone)

#         return backbone, c, b

# model.backbone = BrainNetwork(h=hidden_dim, in_dim=hidden_dim, seq_len=seq_len, 
#                         clip_size=clip_emb_dim, out_dim=clip_emb_dim*clip_seq_dim) 
# utils_mindeye.count_params(model.ridge)
# utils_mindeye.count_params(model.backbone)
# utils_mindeye.count_params(model)

# # setup diffusion prior network
# out_dim = clip_emb_dim
# depth = 6
# dim_head = 52
# heads = clip_emb_dim//52 # heads * dim_head = clip_emb_dim
# timesteps = 100

# prior_network = PriorNetwork(
#         dim=out_dim,
#         depth=depth,
#         dim_head=dim_head,
#         heads=heads,
#         causal=False,
#         num_tokens = clip_seq_dim,
#         learned_query_mode="pos_emb"
#     )

# model.diffusion_prior = BrainDiffusionPrior(
#     net=prior_network,
#     image_embed_dim=out_dim,
#     condition_on_text_encodings=False,
#     timesteps=timesteps,
#     cond_drop_prob=0.2,
#     image_embed_scale=None,
# )
# model.to(device)

# utils_mindeye.count_params(model.diffusion_prior)
# utils_mindeye.count_params(model)

In [5]:
# # Load pretrained model ckpt
# # Replace with pre_trained_fine_tuned_model.pth
# # tag='pretrained_fine-tuned_sliceTimed0.5.pth'
# # tag='pretrained_fine-tuned_sliceTimed.pth'
# outdir = f'{data_path}/model'

# # print(f"\n---loading {outdir}/{tag}.pth ckpt---\n")
# # try:
# checkpoint = torch.load(outdir+f'/{model_name}.pth', map_location='cpu')
# state_dict = checkpoint['model_state_dict']
# model.load_state_dict(state_dict, strict=True)
# del checkpoint
# # except: # probably ckpt is saved using deepspeed format
# #     import deepspeed
# #     state_dict = deepspeed.utils.zero_to_fp32.get_fp32_state_dict_from_zero_checkpoint(checkpoint_dir=outdir, tag=tag)
# #     model.load_state_dict(state_dict, strict=False)
# #     del state_dict
# # print("ckpt loaded!")


In [6]:
# # prep unCLIP
# config = OmegaConf.load(f"{project_path}/models/generative_models/configs/unclip6.yaml")
# config = OmegaConf.to_container(config, resolve=True)
# unclip_params = config["model"]["params"]
# network_config = unclip_params["network_config"]
# denoiser_config = unclip_params["denoiser_config"]
# # first_stage_config = unclip_params["first_stage_config"]
# conditioner_config = unclip_params["conditioner_config"]
# sampler_config = unclip_params["sampler_config"]
# scale_factor = unclip_params["scale_factor"]
# disable_first_stage_autocast = unclip_params["disable_first_stage_autocast"]
# offset_noise_level = unclip_params["loss_fn_config"]["params"]["offset_noise_level"]
# # first_stage_config['target'] = 'sgm.models.autoencoder.AutoencoderKL'
# sampler_config['params']['num_steps'] = 38
# with open(f"{storage_path}/diffusion_engine", "rb") as input_file:
#     diffusion_engine = pickle.load(input_file)
# # set to inference
# diffusion_engine.eval().requires_grad_(False)
# diffusion_engine.to(device)
# ckpt_path = f'{cache_dir}/unclip6_epoch0_step110000.ckpt'
# ckpt = torch.load(ckpt_path, map_location='cpu')
# diffusion_engine.load_state_dict(ckpt['state_dict'])
# batch={"jpg": torch.randn(1,3,1,1).to(device), # jpg doesnt get used, it's just a placeholder
#     "original_size_as_tuple": torch.ones(1, 2).to(device) * 768,
#     "crop_coords_top_left": torch.zeros(1, 2).to(device)}
# out = diffusion_engine.conditioner(batch)
# vector_suffix = out["vector"].to(device)
# # f = h5py.File(f'{storage_path}/coco_images_224_float16.hdf5', 'r')
# # images = f['images']

In [7]:
sub = "sub-005"
session = "ses-03"
task = 'C'  # 'study' or 'A'; used to search for functional run in bids format
func_task_name = 'C'  # 'study' or 'A'; used to search for functional run in bids format
n_runs = 11

ses_list = [session]
design_ses_list = [session]
    
task_name = f"_task-{task}" if task != 'study' else ''
designdir = f"{data_path}/events"

In [8]:
data, starts, images, is_new_run, image_names, unique_images, len_unique_images = utils_mindeye.load_design_files(
    sub=sub,
    session=session,
    func_task_name=task,
    designdir=designdir,
    design_ses_list=design_ses_list
)

if sub == 'sub-001':
    if session == 'ses-01':
        assert image_names[0] == 'images/image_686_seed_1.png'
    elif session in ('ses-02', 'all'):
        assert image_names[0] == 'all_stimuli/special515/special_40840.jpg'
    elif session == 'ses-03':
        assert image_names[0] == 'all_stimuli/special515/special_69839.jpg'
    elif session == 'ses-04':
        assert image_names[0] == 'all_stimuli/rtmindeye_stimuli/image_686_seed_1.png'
elif sub == 'sub-003':
    assert image_names[0] == 'all_stimuli/rtmindeye_stimuli/image_686_seed_1.png'

unique_images = np.unique(image_names.astype(str))
unique_images = unique_images[(unique_images!="nan")]
len_unique_images = len(unique_images)

print("n_runs",n_runs)

if (sub == 'sub-001' and session == 'ses-04') or (sub == 'sub-003' and session == 'ses-01'):
    assert len(unique_images) == 851

print(image_names[:4])
print(starts[:4])
print(is_new_run[:4])

image_idx = np.array([])  # contains the unique index of each presented image
vox_image_names = np.array([])  # contains the names of the images corresponding to image_idx
all_MST_images = dict()
for i, im in enumerate(image_names):
    # skip if blank, nan
    if im == "blank.jpg":
        i+=1
        continue
    if str(im) == "nan":
        i+=1
        continue
    vox_image_names = np.append(vox_image_names, im)
            
    image_idx_ = np.where(im==unique_images)[0].item()
    image_idx = np.append(image_idx, image_idx_)
    
    all_MST_images[i] = im
    i+=1
    
image_idx = torch.Tensor(image_idx).long()
# for im in new_image_names[MST_images]:
#     assert 'MST_pairs' in im
# assert len(all_MST_images) == 300

unique_MST_images = np.unique(list(all_MST_images.values())) 

MST_ID = np.array([], dtype=int)

vox_idx = np.array([], dtype=int)
j=0  # this is a counter keeping track of the remove_random_n used later to index vox based on the removed images; unused otherwise
for i, im in enumerate(image_names):  # need unique_MST_images to be defined, so repeating the same loop structure
    # skip if blank, nan
    if im == "blank.jpg":
        i+=1
        continue
    if str(im) == "nan":
        i+=1
        continue
    j+=1
    curr = np.where(im == unique_MST_images)
    # print(curr)
    if curr[0].size == 0:
        MST_ID = np.append(MST_ID, np.array(len(unique_MST_images)))  # add a value that should be out of range based on the for loop, will index it out later
    else:
        MST_ID = np.append(MST_ID, curr)
        
assert len(MST_ID) == len(image_idx)
# assert len(np.argwhere(pd.isna(data['current_image']))) + len(np.argwhere(data['current_image'] == 'blank.jpg')) + len(image_idx) == len(data)
# MST_ID = torch.tensor(MST_ID[MST_ID != len(unique_MST_images)], dtype=torch.uint8)  # torch.tensor (lowercase) allows dtype kwarg, Tensor (uppercase) is an alias for torch.FloatTensor
print(MST_ID.shape)
if (sub == 'sub-001' and session == 'ses-04') or (sub == 'sub-003' and session == 'ses-01'):
    assert len(all_MST_images) == 100

Data shape: (780, 126)
Using design file: /teamspace/gcs_folders/share/real_time_mindeye_data/3t_data/data/events/csv/sub-005_ses-03.csv
Total number of images: 770
Number of unique images: 532
n_runs 11
['all_stimuli/unchosen_nsd_1000_images/unchosen_7211_cocoid_59250.png'
 'all_stimuli/special515/special_67295.jpg'
 'all_stimuli/unchosen_nsd_1000_images/unchosen_5729_cocoid_53029.png'
 'all_stimuli/special515/special_70232.jpg']
[174.7109683 178.7049172 182.7072832 186.7297016]
[0. 0. 0. 0.]
(693,)


In [9]:
import imageio.v2 as imageio
resize_transform = transforms.Resize((224, 224))
MST_images = []
images = None
for im_name in tqdm(image_idx):
    image_file = f"{unique_images[im_name]}"
    im = imageio.imread(f"{data_path}/{image_file}")
    im = torch.Tensor(im / 255).permute(2,0,1)
    im = resize_transform(im.unsqueeze(0))
    if images is None:
        images = im
    else:
        images = torch.vstack((images, im))
    if ("MST_pairs" in image_file): # ("_seed_" not in unique_images[im_name]) and (unique_images[im_name] != "blank.jpg") 
        MST_images.append(True)
    else:
        MST_images.append(False)

print("images", images.shape)
MST_images = np.array(MST_images)
print("len MST_images", len(MST_images))
if not (sub == 'sub-005' and session == 'ses-06'):
    assert len(MST_images[MST_images==True]) == 124
print("MST_images==True", len(MST_images[MST_images==True]))


  0%|          | 0/693 [00:00<?, ?it/s]

  0%|          | 1/693 [00:01<16:26,  1.43s/it]/teamspace/studios/this_studio/rtcloud-projects/mindeye/conf/.venv/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|██████████| 693/693 [02:35<00:00,  4.45it/s]

images torch.Size([693, 3, 224, 224])
len MST_images 693
MST_images==True 124


In [10]:
def get_image_pairs(sub, session, func_task_name, designdir):
    """Loads design files and processes image pairs for a given session."""
    _, _, _, _, image_names, unique_images, _ = utils_mindeye.load_design_files(
        sub=sub,
        session=session,
        func_task_name=func_task_name,
        designdir=designdir,
        design_ses_list=[session]  # Ensure it's a list
    )
    return utils_mindeye.process_images(image_names, unique_images)

In [11]:
from collections import defaultdict

all_dicts = []
for s_idx, s in enumerate(ses_list):
    im, vo, _ = get_image_pairs(sub, s, func_task_name, designdir)
    assert len(im) == len(vo)
    all_dicts.append({k:v for k,v in enumerate(vo)})

image_to_indices = defaultdict(lambda: [[] for _ in range(len(ses_list))])
for ses_idx, idx_to_name in enumerate(all_dicts):
    for idx, name in idx_to_name.items():
        image_to_indices[name][ses_idx].append(idx)
        
image_to_indices = dict(image_to_indices)

Data shape: (780, 126)
Using design file: /teamspace/gcs_folders/share/real_time_mindeye_data/3t_data/data/events/csv/sub-005_ses-03.csv
Total number of images: 770
Number of unique images: 532


In [12]:
utils_mindeye.seed_everything(0)
MST_idx = [v[0][0] if len(v[0]) > 0 else None for k, v in image_to_indices.items() if 'MST_pairs' in k]
# Remove any None values (in case some images don't have repeats)
MST_idx = [idx for idx in MST_idx if idx is not None]

print("MST_idx", len(MST_idx))

MST_idx 62


In [13]:
import pandas as pd
import nibabel as nib
from nilearn.glm.first_level import FirstLevelModel
from nilearn.image import get_data, index_img, concat_imgs, new_img_like

# get the mask and the reference files
ndscore_events = [pd.read_csv(f'{data_path}/events/{sub}_{session}_task-{func_task_name}_run-{run+1:02d}_events.tsv', sep = "\t", header = 0) for run in range(n_runs)]  # create a new list of events_df's which will have the trial_type modified to be unique identifiers
ndscore_tr_labels = [pd.read_csv(f"{data_path}/events/{sub}_{session}_task-{func_task_name}_run-{run+1:02d}_tr_labels.csv") for run in range(n_runs)]
tr_length = 1.5
mask_img = nib.load(f'{data_path}/{sub}_final_mask.nii.gz')  # nsdgeneral mask in functional space
assert sub == 'sub-005'
fmriprep_boldref_path = f"{data_path}/sub-005_ses-01_task-C_run-01_space-T1w_boldref.nii.gz"  # preprocessed boldref from ses-01
single_vols_path = f"{derivatives_path}/vols/{sub}/{session}"
os.makedirs(single_vols_path, exist_ok=True)
rt_vol0_path = os.path.join(single_vols_path, f"{sub}_{session}_task-{func_task_name}_run-01_bold_0000.nii.gz") # first volume (vol0000) of real-time session

def fast_apply_mask(target=None,mask=None):
    return target[np.where(mask == 1)].T
fmriprep_boldref_nib = nib.load(fmriprep_boldref_path)

# union_mask = np.load(f"{data_path}/union_mask_from_ses-01-02.npy")
rel_mask = np.load(f'/teamspace/studios/this_studio/mindeye_offline/sub-005_ses-01_task-C_relmask.npy')

In [14]:
# apply union mask to the nsdgeneral ROI and convert to nifti
# assert mask_img.get_fdata().sum() == union_mask.shape
assert mask_img.get_fdata().sum() == rel_mask.shape
# union_mask_img = new_img_like(mask_img, union_mask)
union_mask_img = new_img_like(mask_img, rel_mask)

In [15]:
# apply union_mask to mask_img and return nifti object

# Get the data as a boolean array
mask_data = mask_img.get_fdata().astype(bool)

# Flatten only the True voxels in the mask
true_voxel_indices = np.where(mask_data.ravel())[0]

# Apply the union_mask (boolean mask of size 19174)
# selected_voxel_indices = true_voxel_indices[union_mask]
selected_voxel_indices = true_voxel_indices[rel_mask]

# Create a new flattened mask with all False
new_mask_flat = np.zeros(mask_data.size, dtype=bool)

# Set selected voxels to True
new_mask_flat[selected_voxel_indices] = True

# Reshape back to original 3D shape
new_mask_data = new_mask_flat.reshape(mask_data.shape)

# Create new NIfTI image
union_mask_img = nib.Nifti1Image(new_mask_data.astype(np.uint8), affine=mask_img.affine)

In [16]:
print("union_mask_img.shape", union_mask_img.shape)
print("union mask num voxels", int(union_mask_img.get_fdata().sum()))

union_mask_img.shape (76, 90, 74)
union mask num voxels 2792


In [17]:
def do_reconstructions(betas_tt):
    """
    takes in the beta map for a stimulus trial in torch tensor format (tt)

    returns reconstructions and clipvoxels for retrievals
    """
    # start_reconstruction_time = time.time()
    model.to(device)
    model.eval().requires_grad_(False)
    clipvoxelsTR = None
    reconsTR = None
    num_samples_per_image = 1
    with torch.no_grad(), torch.cuda.amp.autocast(dtype=torch.float16):
        voxel = betas_tt
        voxel = voxel.to(device)
        voxel_ridge = model.ridge(voxel[:,[0]],0) # 0th index of subj_list
        backbone0, clip_voxels0, blurry_image_enc0 = model.backbone(voxel_ridge)
        clip_voxels = clip_voxels0
        backbone = backbone0
        blurry_image_enc = blurry_image_enc0[0]
        clipvoxelsTR = clip_voxels.cpu()
        prior_out = model.diffusion_prior.p_sample_loop(backbone.shape, 
                        text_cond = dict(text_embed = backbone), 
                        cond_scale = 1., timesteps = 20)  
        for i in range(len(voxel)):
            samples = utils_mindeye.unclip_recon(prior_out[[i]],
                            diffusion_engine,
                            vector_suffix,
                            num_samples=num_samples_per_image)
            if reconsTR is None:
                reconsTR = samples.cpu()
            else:
                reconsTR = torch.vstack((reconsTR, samples.cpu()))
            imsize = 224
            reconsTR = transforms.Resize((imsize,imsize), antialias=True)(reconsTR).float().numpy().tolist()
        return reconsTR, clipvoxelsTR
    
def batchwise_cosine_similarity(Z,B):
    Z = Z.flatten(1)
    B = B.flatten(1).T
    Z_norm = torch.linalg.norm(Z, dim=1, keepdim=True)  # Size (n, 1).
    B_norm = torch.linalg.norm(B, dim=0, keepdim=True)  # Size (1, b).
    cosine_similarity = ((Z @ B) / (Z_norm @ B_norm)).T
    return cosine_similarity

def get_top_retrievals(clipvoxel, all_images, total_retrievals = 1):
    '''
    clipvoxel: output from do_recons that contains that information needed for retrievals
    all_images: all ground truth actually seen images by the participant in day 2 run 1

    outputs the top retrievals
    '''
    values_dict = {}
    with torch.cuda.amp.autocast(dtype=torch.float16):
        emb = clip_img_embedder(torch.reshape(all_images,(all_images.shape[0], 3, 224, 224)).to(device)).float() # CLIP-Image
        emb = emb.cpu()
        emb_ = clipvoxel # CLIP-Brain
        emb = emb.reshape(len(emb),-1)
        emb_ = np.reshape(emb_, (1, 425984))
        emb = nn.functional.normalize(emb,dim=-1)
        emb_ = nn.functional.normalize(emb_,dim=-1)
        emb_ = emb_.float()
        fwd_sim = batchwise_cosine_similarity(emb_,emb)  # brain, clip
        print("Given Brain embedding, find correct Image embedding")
    fwd_sim = np.array(fwd_sim.cpu())
    which = np.flip(np.argsort(fwd_sim, axis = 0))
    imsize = 224
    for attempt in range(total_retrievals):
        values_dict[f"attempt{(attempt+1)}"] = transforms.Resize((imsize,imsize), antialias=True)(all_images[which[attempt].copy()]).float().numpy().tolist()
    return values_dict


def convert_image_array_to_PIL(image_array):
    if image_array.ndim == 4:
        image_array = image_array[0]

    # get the dimension to h, w, 3|1
    if image_array.ndim == 3 and image_array.shape[0] == 3:
        image_array = np.transpose(image_array, (1, 2, 0))  # Change shape to (height, width, 3)
    
    # clip the image array to 0-1
    image_array = np.clip(image_array, 0, 1)
    # convert the image array to uint8
    image_array = (image_array * 255).astype('uint8')
    # convert the image array to PIL
    return Image.fromarray(image_array)


In [18]:
# from utils_glm import load_glmsingle_hrf_library, hrf_i_factory, fit_and_run_glm
# BASE_TIME, GLMS_HRFS = load_glmsingle_hrf_library(f"{data_path}/getcanonicalhrflibrary.tsv")
# hrf_fns = [hrf_i_factory(i, BASE_TIME, GLMS_HRFS) for i in range(1, 21)]
# hrf_indices = np.load(f"{data_path}/avg_hrfs_s1_s2_full.npy").astype(int)[:,:,:,0]

In [19]:
plot_images=True
save_individual_images=False
only_betas = True  # skip plotting and saving, only calculate betas (located in all_betas)

In [20]:
local_derivatives_path = './local_derivatives'
os.makedirs(local_derivatives_path, exist_ok=True)

In [21]:
import subprocess, os
bash_env = subprocess.run("bash -c 'source ~/.bashrc; env'", capture_output=True, text=True, shell=True)
for line in bash_env.stdout.splitlines():
    key, _, value = line.partition("=")
    os.environ[key] = value

In [22]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)


In [23]:
def compute_affine_transform(moving_img, reference_img, output_mat_path, dof=6):
    """Compute affine transform matrix aligning moving_img to reference_img using FSL FLIRT."""
    """moving_img and reference_img can be paths to nifti files or nibabel objects"""
    moving_img_path = moving_img if isinstance(moving_img, str) else moving_img.filename
    reference_img_path = reference_img if isinstance(reference_img, str) else reference_img.filename
    
    cmd = f"flirt -in {moving_img_path} \
        -ref {reference_img_path} \
        -omat {output_mat_path} \
        -dof {dof}"
    os.system(cmd)
    return moving_img_path, reference_img_path, output_mat_path

def motion_correct_volume(input_vol_path, ref_vol_path, output_prefix):
    """
    Run FSL mcflirt to motion-correct input_vol to ref_vol.
    Returns paths to the motion parameter file and matrix folder.
    """
    cmd = f"mcflirt -in {input_vol_path} -reffile {ref_vol_path} -out {output_prefix} -plots -mats"
    os.system(cmd)
    par_file = f"{output_prefix}.par"
    mats_dir = f"{output_prefix}.mat"
    return par_file, mats_dir

def concatenate_fsl_transforms(first_mat, second_mat, output_mat):
    """
    Concatenate two affine transforms using FSL convert_xfm.
    Order matters: result = second_mat ∘ first_mat
    """
    cmd = f"convert_xfm -concat {second_mat} -omat {output_mat} {first_mat}"
    os.system(cmd)
    return output_mat

def apply_fsl_transform(input_vol, ref_vol, output_vol, transform_mat):
    """
    Apply a precomputed affine transform to resample input_vol into ref_vol space.
    """
    cmd = f"flirt -in {input_vol} -ref {ref_vol} -out {output_vol} -init {transform_mat} -applyxfm"
    os.system(cmd)
    return output_vol

In [ ]:
for delay in tqdm((1,3,5,10,15,20)):
    redo_motion_correct = False # turn this to true if this is the first time running it for a session
    motion_corrected_dir = f"{local_derivatives_path}/motion_corrected"
    motion_corrected_resampled_dir = f"{local_derivatives_path}/motion_corrected_resampled"

    # prepare output dirs
    if redo_motion_correct:
        if os.path.exists(motion_corrected_dir):
            shutil.rmtree(motion_corrected_dir)
        if os.path.exists(motion_corrected_resampled_dir):
            shutil.rmtree(motion_corrected_resampled_dir)
        
        os.makedirs(motion_corrected_dir, exist_ok=True)
        os.makedirs(motion_corrected_resampled_dir, exist_ok=True)
        os.makedirs(f"{local_derivatives_path}/refs", exist_ok=True)
    else:
        print("Motion correction already done, skipping motion correction")

    # make sure FSL writes NIFTI_GZ
    rt_to_fmriprep_mat_path = f"{local_derivatives_path}/rt_to_fmriprep_mat"
    os.environ["FSLOUTPUTTYPE"] = "NIFTI_GZ"

    assert np.all(fmriprep_boldref_nib.affine == union_mask_img.affine)

    all_betas_all_runs = []
    all_cropped_events = []
    n_runs = 11
    # delay = 0  # arbitrary delay in REAL trials (not including blank.jpg) this is defined on the outside loop
    num_TRs = 192

    for run_num in range(1, n_runs + 1):
        # get the events
        events_all = ndscore_events[run_num - 1].copy()
        events_all["onset"] = events_all["onset"].astype(float)
        run_start_time = events_all["onset"].iloc[0]
        events_all["onset"] -= run_start_time  # make first onset start at 0

        # drop blank.jpg trials from the GLM, I tried including them as trials to fit and correlation with GLM single is worse
        nonblank_mask = events_all["image_name"] != "blank.jpg"
        events_nonblank = events_all[nonblank_mask].copy().reset_index(drop=True)
        events_nonblank["trial_number"] = events_nonblank["trial_number"].astype(int)

        # real trial_numbers in order (only non-blank.jpg)
        real_trial_numbers = events_nonblank["trial_number"].tolist()
        n_real_trials = len(real_trial_numbers)

        # get HRF labels for the current run
        tr_labels_hrf = ndscore_tr_labels[run_num - 1]["tr_label_hrf"].tolist()

        assert len(tr_labels_hrf) == num_TRs, "There should be image labels for each TR"
        assert all(
            (label in image_names) or (label in ("blank", "blank.jpg"))
            for label in tr_labels_hrf
        ), "Some labels in tr_labels_hrf are missing from image_names (or not blank/blank.jpg)."

        # mcparams / data containers for this run
        all_run_motion_corrections_params = []
        all_run_vols = []

        # betas container for this run only
        all_betas_run = []

        # as safety to avoid duplicates
        reconstructed_real_indices = set()

        # counter over trials (not blank.jpg) as they reach their HRF-event TR
        real_hrf_counter = 0

        for TR in range(num_TRs):
            # Get the volume
            if redo_motion_correct:
                cur_vol_name = f"{sub}_{session}_task-{func_task_name}_run-{run_num:02d}_bold_{TR:04d}"
                cur_vol_path = os.path.join(single_vols_path, f"{cur_vol_name}.nii.gz")

                if TR == 0 and run_num == 1:
                    # Save first rt volume and compute affine transform to fmriprep boldref for future volumes
                    cur_vol_data = nib.load(cur_vol_path)
                    nib.save(cur_vol_data, rt_vol0_path)
                    _ = compute_affine_transform(
                        rt_vol0_path, fmriprep_boldref_path, rt_to_fmriprep_mat_path
                    )

                motion_corrected_vol_path = f"{motion_corrected_dir}/{cur_vol_name}_mc"
                motion_corrected_params_path, motion_corrected_mat_path = motion_correct_volume(
                    cur_vol_path, rt_vol0_path, motion_corrected_vol_path
                )

                current_TR_to_session_ref = (
                    f"{local_derivatives_path}/refs/current_TR_to_session_ref_run{run_num}_{TR:04d}"
                )
                final_transform_mat_path = concatenate_fsl_transforms(
                    f"{motion_corrected_mat_path}/MAT_0000",
                    rt_to_fmriprep_mat_path,
                    current_TR_to_session_ref,
                )
                final_TR_vol_path = (
                    f"{motion_corrected_resampled_dir}/{session}_run-{run_num:02d}_{TR:04d}_mc_boldres.nii.gz"
                )
                preprocessed_TR_path = apply_fsl_transform(
                    cur_vol_path,
                    fmriprep_boldref_path,
                    final_TR_vol_path,
                    final_transform_mat_path,
                )

                shutil.rmtree(motion_corrected_mat_path)
                all_run_motion_corrections_params.append(
                    np.loadtxt(motion_corrected_params_path)
                )
                all_run_vols.append(get_data(preprocessed_TR_path))

            else:
                # Just load them to avoid wasting time.
                c_final_vol_path = (
                    f"{motion_corrected_resampled_dir}/{session}_run-{run_num:02d}_{TR:04d}_mc_boldres.nii.gz"
                )
                cur_vol_name = (
                    f"{sub}_{session}_task-{func_task_name}_run-{run_num:02d}_bold_{TR:04d}"
                )
                c_motion_corrected_params_path = (
                    f"{motion_corrected_dir}/{cur_vol_name}_mc.par"
                )

                all_run_motion_corrections_params.append(
                    np.loadtxt(c_motion_corrected_params_path)
                )
                all_run_vols.append(get_data(c_final_vol_path))


            current_label = tr_labels_hrf[TR]

            print(
                "Real HRF counter:", real_hrf_counter,
                "TR:", TR,
                "label:", current_label
            )

            # 'blank' means no HRF event at this TR and 'blank.jpg' is also excluded as trial.
            if current_label in ("blank", "blank.jpg"):
                continue

            # this shouldn't happen but if there's more hrf labels as trials than events as trials trigger an error.
            if real_hrf_counter >= n_real_trials:
                raise ValueError(
                    f"real_hrf_counter {real_hrf_counter} exceeds number of real trials {n_real_trials}"
                )

            # get the trial number for this hrf, this should be the same if we wouldn't exclude blank.jpg
            trial_number_at_this_hrf = real_trial_numbers[real_hrf_counter]

            # start fitting glm if we have enough trials
            if real_hrf_counter >= delay:
                rec_index = real_hrf_counter - delay  # index into real_trial_numbers
                
                if rec_index < 0 or rec_index >= n_real_trials:
                    # This shouldn't happen, lol.
                    raise ValueError(
                        f"rec_index {rec_index} is out of bounds for n_real_trials {n_real_trials}"
                    )

                if rec_index in reconstructed_real_indices:
                    # This shouln't happen neither, lol.
                    raise ValueError(
                        f"rec_index {rec_index} somehow has already been reconstructed/glm_fitted"
                    )

                trial_number_to_reconstruct = real_trial_numbers[rec_index]

                current_time = TR * tr_length
                cropped_events = events_nonblank[
                    events_nonblank["onset"] <= current_time
                ].copy()

                cropped_events.loc[:, "trial_type"] = np.where(
                    cropped_events["trial_number"] == trial_number_to_reconstruct,
                    "probe",
                    "reference",
                )
                cropped_events = cropped_events.drop(
                    columns=["is_correct", "image_name", "response_time", "trial_number"]
                )
                all_cropped_events.append(cropped_events)

                # get 4D run image from all_run_vols seen so far
                run_brain_imgs = np.rollaxis(np.array(all_run_vols), 0, 4)
                run_brain_imgs = new_img_like(
                    fmriprep_boldref_nib, run_brain_imgs, copy_header=True
                )

                # fit LSS GLM
                lss_glm = FirstLevelModel(
                    t_r=tr_length,
                    slice_time_ref=0,
                    hrf_model="glover",
                    drift_model="cosine",
                    drift_order=1,
                    high_pass=0.01,
                    mask_img=union_mask_img,
                    signal_scaling=False,
                    smoothing_fwhm=None,
                    noise_model="ar1",
                    n_jobs=-1,
                    verbose=-1,
                    memory_level=1,
                    minimize_memory=True,
                )

                lss_glm.fit(
                    run_imgs=run_brain_imgs,
                    events=cropped_events,
                    confounds=pd.DataFrame(
                        np.array(all_run_motion_corrections_params)
                    ),
                )

                # get beta map
                beta_map = lss_glm.compute_contrast("probe", output_type="effect_size")
                beta_map_np = beta_map.get_fdata()
                beta_map_np = fast_apply_mask(
                    target=beta_map_np, mask=union_mask_img.get_fdata()
                )
                all_betas_run.append(beta_map_np)
                reconstructed_real_indices.add(rec_index)

            # increase counter
            real_hrf_counter += 1

        # at the end of the TR loop, fit the GLM for remaining trials when delay > 0
        remaining_indices = [
            idx for idx in range(n_real_trials) if idx not in reconstructed_real_indices
        ]

        if remaining_indices:
            # Build full-run brain image and confounds once
            full_run_brain_imgs = np.rollaxis(np.array(all_run_vols), 0, 4)
            full_run_brain_imgs = new_img_like(
                fmriprep_boldref_nib, full_run_brain_imgs, copy_header=True
            )
            full_confounds = pd.DataFrame(
                np.array(all_run_motion_corrections_params)
            )

            last_time = (num_TRs - 1) * tr_length

            for rec_index in remaining_indices:
                trial_number_to_reconstruct = real_trial_numbers[rec_index]

                cropped_events = events_nonblank[
                    events_nonblank["onset"] <= last_time
                ].copy()

                cropped_events.loc[:, "trial_type"] = np.where(
                    cropped_events["trial_number"] == trial_number_to_reconstruct,
                    "probe",
                    "reference",
                )
                cropped_events = cropped_events.drop(
                    columns=["is_correct", "image_name", "response_time", "trial_number"]
                )

                lss_glm = FirstLevelModel(
                    t_r=tr_length,
                    slice_time_ref=0,
                    hrf_model="glover",
                    drift_model="cosine",
                    drift_order=1,
                    high_pass=0.01,
                    mask_img=union_mask_img,
                    signal_scaling=False,
                    smoothing_fwhm=None,
                    noise_model="ar1",
                    n_jobs=-1,
                    verbose=-1,
                    memory_level=1,
                    minimize_memory=True,
                )

                lss_glm.fit(
                    run_imgs=full_run_brain_imgs,
                    events=cropped_events,
                    confounds=full_confounds,
                )

                beta_map = lss_glm.compute_contrast("probe", output_type="effect_size")
                beta_map_np = beta_map.get_fdata()
                beta_map_np = fast_apply_mask(
                    target=beta_map_np, mask=union_mask_img.get_fdata()
                )
                all_betas_run.append(beta_map_np)
                reconstructed_real_indices.add(rec_index)

        # Save betas for current run only
        os.makedirs("./final_betas", exist_ok=True)
        save_path = f"./final_betas/{session}_run{run_num:02d}_delay{delay}.npy"
        np.save(save_path, np.array(all_betas_run))

        # Append to all runs container
        all_betas_all_runs.extend(all_betas_run)

    # Save betas across all runs
    np.save(
        f"./final_betas/all_betas_all_runs_delay{delay}.npy",
        np.array(all_betas_all_runs)
    )


  0%|          | 0/6 [00:00<?, ?it/s]

Motion correction already done, skipping motion correction
Real HRF counter: 0 TR: 0 label: blank
Real HRF counter: 0 TR: 1 label: blank
Real HRF counter: 0 TR: 2 label: blank
Real HRF counter: 0 TR: 3 label: blank
Real HRF counter: 0 TR: 4 label: all_stimuli/unchosen_nsd_1000_images/unchosen_7211_cocoid_59250.png
Real HRF counter: 1 TR: 5 label: blank
Real HRF counter: 1 TR: 6 label: blank
Real HRF counter: 1 TR: 7 label: all_stimuli/special515/special_67295.jpg


/teamspace/studios/this_studio/rtcloud-projects/mindeye/conf/.venv/lib/python3.11/site-packages/nilearn/glm/model.py:207: RuntimeWarning: invalid value encountered in sqrt
  sd = np.sqrt(self.vcov(matrix=matrix, dispersion=dispersion))


Real HRF counter: 2 TR: 8 label: blank
Real HRF counter: 2 TR: 9 label: blank
Real HRF counter: 2 TR: 10 label: all_stimuli/unchosen_nsd_1000_images/unchosen_5729_cocoid_53029.png
Real HRF counter: 3 TR: 11 label: blank
Real HRF counter: 3 TR: 12 label: all_stimuli/special515/special_70232.jpg
Real HRF counter: 4 TR: 13 label: blank
Real HRF counter: 4 TR: 14 label: blank
Real HRF counter: 4 TR: 15 label: all_stimuli/unchosen_nsd_1000_images/unchosen_7251_cocoid_26645.png
Real HRF counter: 5 TR: 16 label: blank
Real HRF counter: 5 TR: 17 label: blank
Real HRF counter: 5 TR: 18 label: all_stimuli/special515/special_40721.jpg
Real HRF counter: 6 TR: 19 label: blank
Real HRF counter: 6 TR: 20 label: all_stimuli/unchosen_nsd_1000_images/unchosen_1956_cocoid_70856.png
Real HRF counter: 7 TR: 21 label: blank
Real HRF counter: 7 TR: 22 label: blank
Real HRF counter: 7 TR: 23 label: all_stimuli/unchosen_nsd_1000_images/unchosen_2840_cocoid_40668.png
Real HRF counter: 8 TR: 24 label: blank
Real

/teamspace/studios/this_studio/rtcloud-projects/mindeye/conf/.venv/lib/python3.11/site-packages/nilearn/glm/model.py:207: RuntimeWarning: invalid value encountered in sqrt
  sd = np.sqrt(self.vcov(matrix=matrix, dispersion=dispersion))


...

 67%|██████▋   | 4/6 [1:49:32<54:33, 1636.59s/it]  

Motion correction already done, skipping motion correction
Real HRF counter: 0 TR: 0 label: blank
Real HRF counter: 0 TR: 1 label: blank
Real HRF counter: 0 TR: 2 label: blank
Real HRF counter: 0 TR: 3 label: blank
Real HRF counter: 0 TR: 4 label: all_stimuli/unchosen_nsd_1000_images/unchosen_7211_cocoid_59250.png
Real HRF counter: 1 TR: 5 label: blank
Real HRF counter: 1 TR: 6 label: blank
Real HRF counter: 1 TR: 7 label: all_stimuli/special515/special_67295.jpg
Real HRF counter: 2 TR: 8 label: blank
Real HRF counter: 2 TR: 9 label: blank
Real HRF counter: 2 TR: 10 label: all_stimuli/unchosen_nsd_1000_images/unchosen_5729_cocoid_53029.png
Real HRF counter: 3 TR: 11 label: blank
Real HRF counter: 3 TR: 12 label: all_stimuli/special515/special_70232.jpg
Real HRF counter: 4 TR: 13 label: blank
Real HRF counter: 4 TR: 14 label: blank
Real HRF counter: 4 TR: 15 label: all_stimuli/unchosen_nsd_1000_images/unchosen_7251_cocoid_26645.png
Real HRF counter: 5 TR: 16 label: blank
Real HRF counte

 83%|████████▎ | 5/6 [2:10:59<25:10, 1510.52s/it]

Motion correction already done, skipping motion correction
Real HRF counter: 0 TR: 0 label: blank
Real HRF counter: 0 TR: 1 label: blank
Real HRF counter: 0 TR: 2 label: blank
Real HRF counter: 0 TR: 3 label: blank
Real HRF counter: 0 TR: 4 label: all_stimuli/unchosen_nsd_1000_images/unchosen_7211_cocoid_59250.png
Real HRF counter: 1 TR: 5 label: blank
Real HRF counter: 1 TR: 6 label: blank
Real HRF counter: 1 TR: 7 label: all_stimuli/special515/special_67295.jpg
Real HRF counter: 2 TR: 8 label: blank
Real HRF counter: 2 TR: 9 label: blank
Real HRF counter: 2 TR: 10 label: all_stimuli/unchosen_nsd_1000_images/unchosen_5729_cocoid_53029.png
Real HRF counter: 3 TR: 11 label: blank
Real HRF counter: 3 TR: 12 label: all_stimuli/special515/special_70232.jpg
Real HRF counter: 4 TR: 13 label: blank
Real HRF counter: 4 TR: 14 label: blank
Real HRF counter: 4 TR: 15 label: all_stimuli/unchosen_nsd_1000_images/unchosen_7251_cocoid_26645.png
Real HRF counter: 5 TR: 16 label: blank
Real HRF counte

100%|██████████| 6/6 [2:32:56<00:00, 1529.44s/it]
